<a href="https://colab.research.google.com/github/aebongbing/ESAA/blob/main/%ED%95%B8%EC%A6%88%EC%98%A8_%ED%95%84%EC%82%AC_p_245_271.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import warnings

warnings.filterwarnings('ignore')

# import package

import numpy as np

import os

#5장에서 소개한 moons dataset 불러오기

from sklearn.datasets import make_moons

from sklearn.model_selection import train_test_split

X,y = make_moons(n_samples=100, noise=0.15)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# CH07. 앙상블 학습과 랜덤 포레스트

무작위로 선택된 수천명의 사람에게 복잡한 질문을 하고 얻은 답 > 전문가의 답

-> 대중의 지혜

일련의 예측기(앙상블)로부터 예측을 수집하면 가장 좋은 모델 하나보다 더 좋은 예측을 얻을 수 있음

-> 앙상블 학습

앙상블 학습 알고리즘=앙상블 방법

결정 트리의 앙상블 = 랜덤 포레스트

## 7.1 투표 기반 분류기

여러개의 분류기를 훈련시키고 각 분류기의 예측을 모아서 가장 많이 선택된 클래스를 예측하는 것

 = 직접 투표

각 분류기가 약한 학습기일지라도 충분하게 많고 다양하다면 앙상블은 강한 학습기가 될 수 있다




In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

log_clf=LogisticRegression()
rnd_clf=RandomForestClassifier()
svm_clf=SVC()

voting_clf=VotingClassifier(
    estimators=[('lr',log_clf),('rf',rnd_clf),('svc',svm_clf)],
    voting='hard')
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('rf', RandomForestClassifier()), ('svc', SVC())])

In [11]:
from sklearn.metrics import accuracy_score
for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
  clf.fit(X_train,y_train)
  y_pred=clf.predict(X_test)
  print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.65
RandomForestClassifier 0.9
SVC 0.9
VotingClassifier 0.9


모든 분류기가 클래스의 확률을 예측할 수 있다면 개별 분류기의 예측을 평균 내어 확률이 가장 높은 클래스를 예측할 수 있다. 이를 간접투표 라고 한다

이는 확률이 높은 투표에 비중을 두기 떄문에 직접 투표 방식보다 성능이 높다
voting='soft'로 바꾸고 모든 분류기가 클래스 확률을 추정할 수 있으면 된다
SVC는 기본값에서는 클래스 확률을 제공하지 않으므로 probability 매개변수를 True로 지정해야 한다

## 7.2 배깅과 페이스팅

배깅: 훈련 세트에서 중복을 허용하여 샘플링하는 방식

페이스팅: 중복을 허용하지 않고 샘플링하는 방식

배깅과 페이스팅에서는 같은 훈련 샘플을 여러 개의 예측기에 걸쳐 사용할 수 있습니다. 하지만 배깅만이 한 예측기를 위해 같은 훈련 샘플을 여러번 샘플링할 수 있다

수집함수: 전형적으로 분류일때 -> 통계적 최빈값
          회귀에 대해서는 -> 평균

수집함수를 통과하면 편향과 분산이 모두 감소한다             


### 7.2.1 사이킷런의 배깅과 페이스팅

In [13]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf=BaggingClassifier(
    DecisionTreeClassifier(),n_estimators=500,
    max_samples=50, bootstrap=True, n_jobs=-1)
bag_clf.fit(X_train, y_train)
y_pred=bag_clf.predict(X_test)

앙상블은 비슷한 편향에서 더 작은 분산을 만든다

부트스트래핑은 각 예측기가 학습하는 서브셋에 다양성을 증가시키므로 배깅이 페이스팅보다 편향이 조금 더 높다. 하지만 다양성을 추가한다는 것은 예측기들의 상관관계를 줄이므로 앙상블의 분산을 감소시킨다. 전반적으로 배깅이 더 나은 모델을 만들기 때문에 일반적으로 더 선호된다.

### 7.2.2 oob 평가

배깅을 사용하면 어떤 샘플은 한 예측기를 위해 여러 번 샘플링되고 어떤 것은 전혀 선택되지 않을 수 있다. BaggingClassifier는 기본값으로 중복을 허용하여 훈련 세트의 크기만큼인 m개 샘플을 선택한다. 이는 평균적으로 각 예측기에 훈련 샘플의 63% 정도만 샘플링된다는 것을 의미한다. 선택되지 않은 훈련 샘플의 나머지 37%를 oob샘플이라고 부른다

앙상블의 평가는 각 예측기의 oob평가를 평균하여 얻는다

In [14]:
bag_clf=BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    bootstrap=True,n_jobs=-1,oob_score=True)
bag_clf.fit(X_train, y_train)
bag_clf.oob_score_

0.95

In [16]:
from sklearn.metrics import accuracy_score
y_pred=bag_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.85

In [17]:
bag_clf.oob_decision_function_

array([[0.        , 1.        ],
       [0.17112299, 0.82887701],
       [0.03092784, 0.96907216],
       [1.        , 0.        ],
       [0.01123596, 0.98876404],
       [0.        , 1.        ],
       [1.        , 0.        ],
       [0.99425287, 0.00574713],
       [0.36216216, 0.63783784],
       [0.09467456, 0.90532544],
       [0.19631902, 0.80368098],
       [0.10928962, 0.89071038],
       [1.        , 0.        ],
       [0.02469136, 0.97530864],
       [1.        , 0.        ],
       [0.94708995, 0.05291005],
       [0.        , 1.        ],
       [0.16470588, 0.83529412],
       [0.85365854, 0.14634146],
       [1.        , 0.        ],
       [0.00581395, 0.99418605],
       [0.92746114, 0.07253886],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [0.15606936, 0.84393064],
       [1.        , 0.        ],
       [0.33701657, 0.66298343],
       [0.96296296, 0.03703704],
       [1.        , 0.        ],
       [0.41954023, 0.58045977],
       [0.

## 7.3 랜덤 패치와 랜덤 서브스페이스

- 랜덤 패치 방식: 훈련 특성과 샘플을 모두 샘플링하는 것
- 랜덤 서브스페이스 방식: 훈련 샘플을 모두 사용하고 특성은 샘플링하는 것

## 7.4 랜덤 포레스트

In [18]:
# 500개의 트리로 이루어진 랜덤 포레스트 분류기를 여러 CPU 코어에서 훈련시키는 코드
from sklearn.ensemble import RandomForestClassifier

rnd_clf=RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1)
rnd_clf.fit(X_train, y_train)

y_pred_rf=rnd_clf.predict(X_test)

랜덤포레스트 알고리즘은 트리의 노드를 분할할 때, 전체 특성 중에서 최선의 특성을 찾는 대신 무작위로 선택한 측성 후보 중에서 최적의 특성을 찾는 식으로 무작위성을 더 주입한다 이는 결국 트리를 더욱 다양하게 만들고 편향을 손해보는 대신 분산을 낮추어 전체적으로 더 훌륭한 모델을 만들어낸다

In [19]:
bag_clf=BaggingClassifier(
    DecisionTreeClassifier(max_features='auto', max_leaf_nodes=16),
    n_estimators=500, max_samples=1.0, bootstrap=True, n_jobs=-1
)

### 7.4.1 엑스트라 트리

익스트림 랜덤 트리 (=엑스트라 트리): 보통의 결정 트리처럼 최적의 임곗값을 찾는 대신 후보 특성을 사용해 무작위로 분할한 다음 그중에서 최상의 분할을 선택하는 랜덤 포레스트

사이킷런의 ExtraTreesClassifier 사용

### 7.4.2 특성 중요도

특성 중요도=가중치 평균

In [20]:
from sklearn.datasets import load_iris
iris=load_iris()
rnd_clf=RandomForestClassifier(n_estimators=500, n_jobs=-1)
rnd_clf.fit(iris['data'], iris['target'])
for name, score in zip(iris['feature_names'],rnd_clf.feature_importances_):
  print(name,score)

sepal length (cm) 0.09865436281920223
sepal width (cm) 0.023870460216326238
petal length (cm) 0.4082730944453431
petal width (cm) 0.4692020825191284


## 7.5 부스팅

- 부스팅: 약한 학습기를 여러 개 연결하여 강한 학습기를 만드는 앙상블 방법

### 7.5.1 에이다부스트



In [21]:
from sklearn.ensemble import AdaBoostClassifier

ada_clf=AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1),n_estimators=200,
    algorithm='SAMME.R', learning_rate=0.5)
ada_clf.fit(X_train,y_train)

AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=1),
                   learning_rate=0.5, n_estimators=200)

### 7.5.2 그레이디언트 부스팅

- 그레이디언트 부스팅: 에이다부스트처럼 반복마다 샘플의 가중치를 수정하는 대신 이전 예측기가 만든 잔여 오차에 새로운 예측기를 학습시킴

- 그레이디언트 트리 부스팅 (=그레이디언트 부스티드 회귀)

In [22]:
from sklearn.tree import DecisionTreeRegressor

tree_reg1=DecisionTreeRegressor(max_depth=2)
tree_reg1.fit(X,y)

DecisionTreeRegressor(max_depth=2)

In [23]:
y2=y-tree_reg1.predict(X)
tree_reg2=DecisionTreeRegressor(max_depth=2)
tree_reg2.fit(X,y2)

DecisionTreeRegressor(max_depth=2)

In [25]:
y3=y2-tree_reg2.predict(X)
tree_reg3=DecisionTreeRegressor(max_depth=2)
tree_reg2.fit(X,y3)

DecisionTreeRegressor(max_depth=2)

In [26]:
#새로운 샘플 X_new에 대한 예측을 만들려면
y_pred=sum(tree.predict(X_new) for tree in (tree_reg1, tree_reg2, tree_reg3))

NameError: name 'X_new' is not defined

In [28]:
#사이킷런의 GradientBoostingRegressor를 사용하여 GBRT 앙상블을 간단하게 훈련시키기
from sklearn.ensemble import GradientBoostingRegressor

gbrt=GradientBoostingRegressor(max_depth=2, n_estimators=3, learning_rate=1.0)
gbrt.fit(X,y)

GradientBoostingRegressor(learning_rate=1.0, max_depth=2, n_estimators=3)

learning_rate 매개변수가 각 트리의 기여 정도를 조절한다. 이를 0.1처럼 낮게 설정하면 앙상블을 훈련 세트에 학습시키기 위해 많은 트리가 필요하지만 일반적으로 예측의 성능은 좋아진다 이를 축소라고 한다

In [30]:
#최적의 트리 수를 찾고 이를 이용해 GBRT 앙상블을 훈련시키기
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X_train, X_val, y_train, y_val=train_test_split(X,y)

gbrt=GradientBoostingRegressor(max_depth=2, n_estimators=120)
gbrt.fit(X_train, y_train)

errors=[mean_squared_error(y_val, y_pred) for y_pred in gbrt.staged_predict(X_val)]
bst_n_estimators=np.argmin(errors)+1

gbrt_best=GradientBoostingRegressor(max_depth=2, n_estimators=bst_n_estimators)
gbrt_best.fit(X_train, y_train)

GradientBoostingRegressor(max_depth=2, n_estimators=120)

In [31]:
#조기 종료를 사용하여 트리 수 튜닝하기
gbrt=GradientBoostingRegressor(max_depth=2, warm_start=True)

min_val_error=float('inf')
error_going_up=0
for n_estimators in range(1,120):
  gbrt.n_estimators=n_estimators
  gbrt.fit(X_train, y_train)
  y_pred=gbrt.predict(X_val)
  val_error=mean_squared_error(y_val, y_pred)
  if val_error < min_val_error:
    min_val_error = val_error
    error_going_up=0
  else:
    error_going_up+=1
    if error_going_up == 5:
      break #조기종료

- 확률적 그레이디언트 부스팅: GradientBoostingRegressor는 각 트리가 훈련할 때 사용할 훈련 샘플의 비율을 지정할 수 있는 subsample 매개변수도 지원한다. 예를 들어, subsample=0.25라고 하면 각 트리는 무작위로 선택된 25%의 훈련 샘플로 학습된다. 편향이 높아지는 대신 분산이 낮아지게 된다 또한, 훈련 속도를 상당히 높인다 이를 확룰적 그레이디언트 부스팅이라고 한다.

In [32]:
#최적화된 그레이디언트 부스팅 구현으로 XGBoost 파이썬 라이브러리가 유명
import xgboost

xgb_reg=xgboost.XGBRegressor()
xgb_reg.fit(X_train, y_train)
y_pred=xgb_reg.predict(X_val)

In [33]:
xgb_reg.fit(X_train, y_train, eval_set=[(X_val, y_val)],early_stopping_rounds=2)
y_pred=xgb_reg.predict(X_val)

[0]	validation_0-rmse:0.39790
[1]	validation_0-rmse:0.32234
[2]	validation_0-rmse:0.27390
[3]	validation_0-rmse:0.24487
[4]	validation_0-rmse:0.23225
[5]	validation_0-rmse:0.20815
[6]	validation_0-rmse:0.19177
[7]	validation_0-rmse:0.19002
[8]	validation_0-rmse:0.18149
[9]	validation_0-rmse:0.18103
[10]	validation_0-rmse:0.17676
[11]	validation_0-rmse:0.17331
[12]	validation_0-rmse:0.17367
[13]	validation_0-rmse:0.17154
[14]	validation_0-rmse:0.16931
[15]	validation_0-rmse:0.16942


## 7.6 스태킹

'앙상블에 속한 모든 예측기의 예측을 취합하는 간단한 함수를 사용하는 대신 취합하는 모델을 훈련시킬 수는 없을까'

사이킷런은 스태킹을 직접 지원하지 않지만 직접 구현하는 것은 어렵지 않다